## Loading the data

In [1]:
"""
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
import zipfile, os
with zipfile.ZipFile(os.getcwd() + '/celeba.zip', 'r') as zip_ref:
  zip_ref.extractall(os.getcwd()+'/data_faces')
"""

"\n!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip\nimport zipfile, os\nwith zipfile.ZipFile(os.getcwd() + '/celeba.zip', 'r') as zip_ref:\n  zip_ref.extractall(os.getcwd()+'/data_faces')\n"

# Importing the libraries

In [2]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import os

Using TensorFlow backend.


## Defining the GAN

In [3]:
def define_GAN(g_model, d_model):
    d_model.trainable = False
    model = keras.models.Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = keras.optimizers.adam(learning_rate= 0.0002,
                                beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt)
    return model

## Defining the Discriminator

In [4]:
def define_discriminator(input_shape= (96,80,3)):
    model = keras.models.Sequential()
    
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (3,3),
                                  padding= 'same',
                                  input_shape= input_shape))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))

    # 96 * 80 * 3
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (3,3),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # 48 * 40 * 3
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (4,4),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # 24 * 20 * 3
    model.add(keras.layers.Conv2D(filters= 128,
                                  kernel_size= (4,4),
                                  strides= (2,2),
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.4))
    
    # 12 * 10 * 3

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(units= 1,
                                 activation= 'sigmoid'))
    opt = keras.optimizers.adam(learning_rate= 0.0002, beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt, metrics= ['accuracy'])
    
    return model

## Defining the Generator

In [5]:
def define_generator(latent_dim):
    model = keras.models.Sequential()

    model.add(keras.layers.Dense(units= 256 * 6 * 5, input_dim= latent_dim))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Reshape((6, 5, 256)))
    # 6 * 5
    model.add(keras.layers.Conv2DTranspose(filters= 256,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 12 * 10
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 24 * 20
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 48 * 40
    model.add(keras.layers.Conv2DTranspose(filters= 128,
                                           kernel_size= (4,4),
                                           padding= 'same',
                                           strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # 96 * 80
    
    # Ready for colorization
    model.add(keras.layers.Conv2D(filters= 3,
                                  kernel_size= (3,3),
                                  padding= 'same',
                                  activation= 'tanh'))
    
    return model

## Generating the Fake samples

In [6]:
def generate_latent_points(latent_dim, n_samples):
    X = np.random.randn(latent_dim * n_samples)
    X = X.reshape((n_samples, latent_dim))
    return X

In [7]:
def generate_fake_sample(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim= latent_dim,
                                     n_samples= n_samples)
    
    X = g_model.predict(x_input)
    y = np.zeros((n_samples, 1))
    return X, y

## Generating the Real samples

In [8]:
def generate_real_sample(train_it):
    X, _ = train_it.next()
    X = X[:, 6:102,4:84,:].astype('float32')
    X = (X - 127.5) / 127.5
    y = np.ones((X.shape[0], 1))
    # y = 0.9 * np.ones((n_samples, 1)) + 0.2 * np.random.rand(n_samples).reshape((n_samples, 1))
    return X, y

## Saving the Plot and summarizing

In [9]:
def save_plot(x_input, epoch, n=5):
    x_input = (x_input + 1.0) / 2.0
    filename = f'generated_{epoch + 1}.png'
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.imshow(x_input[i,:,:,:])
        plt.axis('off')
    plt.savefig(filename)
    plt.close()

In [10]:
def summarize_the_model(g_model, d_model, epoch, latent_dim, n_samples, train_iter):
    X_real, y_real = generate_real_sample(train_it= train_iter)
    X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                          latent_dim= latent_dim,
                                          n_samples= n_samples)
    print(f'Accuracy on real data: {d_model.evaluate(X_real, y_real, verbose= 0)}')
    print(f'Accuracy on fake data: {d_model.evaluate(X_fake, y_fake, verbose= 0)}')
    filename = f'model_e_{epoch+1:03}.h5'
    save_plot(x_input= X_fake,
              epoch= epoch)
    checkpnt_path_g = 'drive/My Drive/CGAN/g_model.h5'
    checkpnt_path_d = 'drive/My Drive/CGAN/d_model.h5'
    g_model.save(filename)
    g_model.save(checkpnt_path_g)
    d_model.save(checkpnt_path_d)

# Function for training the GAN

In [11]:
def train_GAN(gan_model, g_model, d_model, dataset_len, latent_dim, train_iter, iters= 100, batch_size= 256):
    half_batch = int(batch_size / 2)
    batch_per_epoch = int(dataset_len / batch_size)
    for i in range(iters):
        for j in range(5):
            X_real, y_real = generate_real_sample(train_it= train_it)
            X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                                  latent_dim= latent_dim,
                                                  n_samples= half_batch)
            X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
            dloss = d_model.train_on_batch(X, y)
            x_gan = generate_latent_points(latent_dim= latent_dim,
                                             n_samples= batch_size)
            y_gan = np.ones((batch_size, 1))
            gloss = gan_model.train_on_batch(x_gan, y_gan)
            print(f'> Epoch: {i+1}, {j+1}/{batch_per_epoch}, dloss: {dloss[0]:5f}, gloss: {gloss:5f}')
        # After each epoch
        summarize_the_model(g_model= g_model,
                            d_model= d_model,
                            epoch= i,
                            latent_dim= latent_dim,
                            n_samples= batch_size, train_iter= train_it)

In [18]:
def load_models(latent_dim, retrain= False):
  if retrain:
    g_model = keras.models.load_model('drive/My Drive/CGAN/g_model.h5')
    d_model = keras.models.load_model('drive/My Drive/CGAN/d_model.h5')
    
    opt = keras.optimizers.adam(learning_rate= 0.0002, beta_1= 0.5)
    d_model.compile(loss= 'binary_crossentropy', optimizer= opt, metrics= ['accuracy'])

    gan_model = define_GAN(g_model=g_model,
                          d_model= d_model)

  else:
    g_model = define_generator(latent_dim= latent_dim)
    d_model = define_discriminator()
    gan_model = define_GAN(g_model=g_model,
                          d_model= d_model)
  return g_model, d_model, gan_model

## Training the Model

In [ ]:
latent_dim = 200
batch_size = 128

g_model, d_model, gan_model = load_models(latent_dim, True)

datagen = keras.preprocessing.image.ImageDataGenerator()
path = os.getcwd() + '/data_faces'
train_it = datagen.flow_from_directory(path,
                                       batch_size= int(batch_size/2),
                                       target_size= (218, 178))


train_GAN(gan_model= gan_model,
          g_model= g_model,
          d_model= d_model,
          dataset_len= 202599,
          latent_dim= latent_dim, train_iter= train_it)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Found 202599 images belonging to 1 classes.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


> Epoch: 1, 1/791, dloss: 0.606556, gloss: 0.685625
> Epoch: 1, 2/791, dloss: 0.700126, gloss: 0.651514
> Epoch: 1, 3/791, dloss: 0.735376, gloss: 0.709210
> Epoch: 1, 4/791, dloss: 0.609544, gloss: 0.952864
> Epoch: 1, 5/791, dloss: 0.464002, gloss: 1.218255
Accuracy on real data: [0.38578368723392487, 0.8125]
Accuracy on fake data: [0.43484701961278915, 1.0]
> Epoch: 2, 1/791, dloss: 0.491807, gloss: 1.109138
> Epoch: 2, 2/791, dloss: 0.423564, gloss: 0.996545
> Epoch: 2, 3/791, dloss: 0.411362, gloss: 0.962451
> Epoch: 2, 4/791, dloss: 0.384854, gloss: 0.966954
> Epoch: 2, 5/791, dloss: 0.376393, gloss: 0.989751
Accuracy on real data: [0.07068948727101088, 0.984375]
Accuracy on fake data: [0.49271493032574654, 1.0]
> Epoch: 3, 1/791, dloss: 0.359993, gloss: 1.011517
> Epoch: 3, 2/791, dloss: 0.326839, gloss: 1.049535
> Epoch: 3, 3/791, dloss: 0.316552, gloss: 1.103641
> Epoch: 3, 4/791, dloss: 0.293582, gloss: 1.167496
> Epoch: 3, 5/791, dloss: 0.277861, gloss: 1.234916
Accuracy on 

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
half_batch = 64
datagen = keras.preprocessing.image.ImageDataGenerator()
path = os.getcwd() + '/data_faces'
train_it = datagen.flow_from_directory(path,
                                       batch_size= int(batch_size/2),
                                       target_size= (218, 178))
d_model = keras.models.load_model('drive/My Drive/CGAN/d_model.h5', compile= False)

opt = keras.optimizers.adam(learning_rate= 0.0002, beta_1= 0.5)
d_model.compile(loss= 'binary_crossentropy', optimizer= opt, metrics= ['accuracy'])

g_model = keras.models.load_model('drive/My Drive/CGAN/g_model.h5')
X_real, y_real = generate_real_sample(train_it= train_it)
X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                      latent_dim= latent_dim,
                                      n_samples= half_batch)
X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
dloss = d_model.train_on_batch(X, y)

Found 202599 images belonging to 1 classes.


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [17]:
dloss

[0.45862672, 0.484375]